<a href="https://colab.research.google.com/github/KexuanZhang/Chinese-Idioms-Translation-with-Neural-Network/blob/main/Davinci_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade openai
!pip install openai_secret_manager

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Loading the dataset

In [ ]:
import json
import urllib.request

In [ ]:
url = 'https://raw.githubusercontent.com/kt2k01/petci/main/data/json/filtered.json'

In [ ]:
response = urllib.request.urlopen(url)
data = json.loads(response.read())

## Preparing the dataset

In [ ]:
data[-2:]

In [ ]:
training_data = []

for idiom in data:
  entry_1 = {}
  entry_2 = {}

  chinese = idiom['chinese']

  if 'gold' in idiom:
    gold = idiom['gold']
    entry_1['prompt'] = chinese + '->'
    entry_1['completion'] = ' ' + gold + '\n'
    training_data.append(entry_1)

  if idiom['human'] != []:
    human = idiom['human'][0]
    entry_2['prompt'] = chinese + '->'
    entry_2['completion'] = ' ' + human + '\n'
    training_data.append(entry_2)

In [ ]:
training_data[:10]

In [ ]:
import random
random.seed(10)

random.shuffle(training_data)

len_train = int(len(training_data )* 0.8)
train_data = training_data[:len_train]
test_data = [x for x in training_data if x not in train_data]
validation_data = train_data[:int(len_train * 0.2)]
train_data = [x for x in train_data if x not in validation_data]

- Check formats of train and validation such that they adhere to OpenAI suggestion

In [ ]:
file_name = "train_data.jsonl"

with open(file_name, "w") as output_file:
  for entry in train_data:
    json.dump(entry, output_file)
    output_file.write("\n")

In [ ]:
!openai tools fine_tunes.prepare_data -f train_data.jsonl

In [ ]:
file_name = "validation_data.jsonl"

with open(file_name, "w") as output_file:
  for entry in validation_data:
    json.dump(entry, output_file)
    output_file.write("\n")

In [ ]:
!openai tools fine_tunes.prepare_data -f validation_data.jsonl

In [ ]:
# the test data doesn't need to be passed to openai since we're testing on them 
# at a different place

file_name = "test_data.jsonl"

with open(file_name, "w") as output_file:
  for entry in test_data:
    json.dump(entry, output_file)
    output_file.write("\n")

## Establish OpenAI's API and training



In [ ]:
import openai

uploading the training file

In [ ]:
file_name = 'train_data.jsonl'

upload_response_train = openai.File.create(
  file=open(file_name, "rb"),
   purpose='fine-tune'
 )
file_id_train = upload_response_train.id
upload_response_train

In [ ]:
file_name = 'validation_data.jsonl'

upload_response_valid = openai.File.create(
  file=open(file_name, "rb"),
   purpose='fine-tune'
 )
file_id_valid = upload_response_valid.id
upload_response_valid

Begin training

In [ ]:
# api_key not provided for secruity
# openai.api_key = ...

In [ ]:
fine_tune_response = openai.FineTune.create(training_file=file_id_train,
                                            validation_file=file_id_valid,
                                            model="davinci")
fine_tune_response

In [ ]:
openai.FineTune.retrieve(fine_tune_response.id)['status']

'pending'

In [ ]:
import time

status = openai.FineTune.retrieve(fine_tune_response.id)['status']
while status != 'succeeded':
  print("current_status: ", status)
  time.sleep(60)
  status = openai.FineTune.retrieve(fine_tune_response.id)['status']


print("current_status: ", status)

In [ ]:
if openai.FineTune.retrieve(fine_tune_response.id)['status'] == 'succeeded':
  fine_tuned_model_id = openai.FineTune.retrieve(fine_tune_response.id)['fine_tuned_model']

##Testing the fine-tune model with examples

In [ ]:
prompt = "心花怒放"

response = openai.Completion.create(
    model=fine_tuned_model_id,
    prompt=prompt,
    max_tokens=15
)

output_text = response.choices[0].text
clean_output = output_text[3:].split("\n", 1)[0]
print(clean_output)

one's heart bursts into full bloom


In [ ]:
prompt = "兴高采烈"

response = openai.Completion.create(
    model=fine_tuned_model_id,
    prompt=prompt,
    max_tokens=15
)

output_text = response.choices[0].text
clean_output = output_text[3:].split("\n", 1)[0]
print(clean_output)

in great fettle


In [ ]:
prompt = "胸有成竹"

response = openai.Completion.create(
    model=fine_tuned_model_id,
    prompt=prompt,
    max_tokens=15
)

output_text = response.choices[0].text
clean_output = output_text[3:].split("\n", 1)[0]
print(clean_output)

have a well-thought-out plan


In [ ]:
prompt = "隔墙有耳"

response = openai.Completion.create(
    model=fine_tuned_model_id,
    prompt=prompt,
    max_tokens=15
)

output_text = response.choices[0].text
clean_output = output_text[3:].split("\n", 1)[0]
print(clean_output)

the walls have ears


In [ ]:
prompt = "顺手牵羊"

response = openai.Completion.create(
    model=fine_tuned_model_id,
    prompt=prompt,
    max_tokens=15
)

output_text = response.choices[0].text
clean_output = output_text[3:].split("\n", 1)[0]
print(clean_output)

lead the goat away from the sheepfold by the neck - filch


In [ ]:
prompt = "一语成谶"

response = openai.Completion.create(
    model=fine_tuned_model_id,
    prompt=prompt,
    max_tokens=15
)

output_text = response.choices[0].text
clean_output = output_text[3:].split("\n", 1)[0]
print(clean_output)

one sentence rhyme and complete the story


In [ ]:
prompt = "齐心协力"

response = openai.Completion.create(
    model=fine_tuned_model_id,
    prompt=prompt,
    max_tokens=15
)

output_text = response.choices[0].text
clean_output = output_text[3:].split("\n", 1)[0]
print(clean_output)

work together with one heart
